In [379]:
import numpy as np
import pandas as pd

hydration_after_p = 0.8
hydration_before_p = 0.045
#Wet Weight(g) after pretreatment
#2% 1h, 12% 1h, 2% 3h 12% 3h
w_60c= np.array([7.8302,4.0316,5.45,4.48]) 
w_80c= np.array([6.4508, 5.181, 5.929, 3.9756]) 
w_100c = np.array([5.6054, 4.1634, 5.9361, 3.9318]) 
w_120c = np.array([4.8918, 3.3319, 5.1648, 2.9348]) 

#Dry Weight(g) after pretreatment
dry_w_60c = w_60c * (1-hydration_after_p)
dry_w_80c = w_80c * (1-hydration_after_p)
dry_w_100c = w_100c * (1-hydration_after_p)
dry_w_120c = w_120c * (1-hydration_after_p)

avg_lost_w_60c = dry_w_60c/2
avg_lost_w_80c = dry_w_80c/2
avg_lost_w_100c = dry_w_100c/2
avg_lost_w_120c = dry_w_120c/2

In [380]:
def calculate_dry_weight(w, hydration_after_p):
    return w * (1 - hydration_after_p)

def calculate_hplc(hplc, k, m, dilution_factor=5):
    return np.array([dilution_factor*(value-m)/k for value in hplc])

def calculate_mass(c, volume_H):
    return c * volume_H

def calculate_max(values, dry_masses):
    return [values[i]*value for i, value in enumerate(dry_masses)]

def calculate_yield(m, max_m):
    return np.divide(m, max_m)

def calculate_average(pairs):
    return np.mean(pairs, axis=1)

def reshape_into_pairs(array):
    return array.reshape(-1, 2)

def format(input_array):
    multiplied_array = input_array * 100
    str_array = np.array(multiplied_array, dtype=str)
    extracted_array = np.array([x[:4] for x in str_array])
    return extracted_array

def format2(input_array):
    str_array = np.array(input_array, dtype=str)
    extracted_array = np.array([x[:4] for x in str_array])
    return extracted_array


In [381]:
def preprocess_data(G_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_pretreatment):

    pairs_G_C = reshape_into_pairs(G_samples_hplc)
    pairs_XMG_C = reshape_into_pairs(XMG_samples_hplc)

    avgs_G_C = calculate_average(pairs_G_C)
    avgs_XMG_C = calculate_average(pairs_XMG_C)
  
    c_G = calculate_hplc(avgs_G_C, k=315718, m=4623)
    c_XMG = calculate_hplc(avgs_XMG_C, k=317400, m=10216)
    
    m_G_before = calculate_mass(c_G, volume)
    m_XMG_before = calculate_mass(c_XMG, volume)

    #Kompensera för att vi inte tog allt när vi gjorde hplc
    wet_mass_into_hplc = 2 #g
    moisture = 0.8
    dried_sample = wet_mass_into_hplc * (1-moisture)
    multiply_factor_over_pretreatment = dry_masses_after_p/dried_sample

    m_G = m_G_before * multiply_factor_over_pretreatment
    m_XMG = m_XMG_before * multiply_factor_over_pretreatment

    #Yield
    yield_glucose_procent_process = calculate_yield(m_G, dry_masses_before_pretreatment)
    yield_XMG_procent_process = calculate_yield(m_XMG, dry_masses_before_pretreatment)
    yield_monosacharides_procent_process = np.add(yield_glucose_procent_process, yield_XMG_procent_process)

    #Selectivity
    selectivity_glucose_procent_process = np.divide(m_G, dry_masses_after_p)
    selectivity_XMG_procent_process = np.divide(m_XMG, dry_masses_after_p)
    selectivity_monosacharices_procent_process = np.divide(m_G+m_XMG, dry_masses_after_p)
    

    results = {
        "Dry W Milled(g)": format2(dry_masses_before_pretreatment),
        "Dry W after Pretreatment(g)": format2(dry_masses_after_p),

        "Glucose(g) ": format2(m_G),
        "XMG(g)": format2(m_XMG),
        "Monosacharides(g)": format2(np.add(m_G, m_XMG)),

        "Selctivity Glucose (%)": format(selectivity_glucose_procent_process),
        "Selctivity XMG (%)": format(selectivity_XMG_procent_process),
        "Selctivity Monosacharides (%)": format(selectivity_monosacharices_procent_process),

        "Yield Glucose (%)": format(yield_glucose_procent_process),
        "Yield XMG (%)": format(yield_XMG_procent_process),
        "Yield Monosacharides (%)": format(yield_monosacharides_procent_process),
    }

    return results

In [382]:
#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#CHANGE THESE
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])

#Constants
glucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100
xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
volume = 0.02485
hydration_after_p = 0.8

dry_masses_before_p = milled_masses*0.955
average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
a = average_dry_masses_before_pretreatment

#CHANGE THIS 
dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])


In [383]:
data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment)


index = ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
process_data_keys = [
    "Dry W Milled(g)",
    "Dry W after Pretreatment(g)",
    "Glucose(g) ",
    "XMG(g)",
    "Monosacharides(g)",
    "Yield Glucose (%)",
    "Yield XMG (%)",
    "Yield Monosacharides (%)",
    "Selctivity Glucose (%)",
    "Selctivity XMG (%)",
    "Selctivity Monosacharides (%)"
]

# Create dataframes
process_data = {key: data[key] for key in process_data_keys}
process_df = pd.DataFrame(process_data)
process_df.index = index

print('2% NaOH batch - Process Data')
print(f"NREL Yield Glucose: {39.54}")
print(f"NREL Yield XMG: {25.24}")
print(f"NREL Yield Monosacharides: {39.54+25.24}")
print()
process_df



2% NaOH batch - Process Data
NREL Yield Glucose: 39.54
NREL Yield XMG: 25.24
NREL Yield Monosacharides: 64.78



,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
120C 1h,1.99,0.97,0.58,0.13,0.71,29.0,6.90,35.9,59.4,14.1,73.5
120C 3h,2.00,1.03,0.68,0.16,0.84,34.1,8.21,42.4,66.2,15.8,82.1
60C 1h,2.00,1.56,0.95,0.23,1.18,47.9,11.5,59.4,61.2,14.6,75.9
60C 3h,2.00,1.09,0.62,0.14,0.77,31.0,7.45,38.5,57.0,13.6,70.6


In [384]:
#Values

#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#2% NaOH 60-120%
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])


dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])

#12% NaOH 60-120% 

#12% NaOH 80-100%

#2% NaOH 80-100%


In [385]:
def create_dataframe(milled_masses, Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, avg_lost, indices, avg_dry_masses_before_p=None):
    # Constants
    hydration_after_p = 0.8
    volume = 0.02485

    
    if avg_dry_masses_before_p is None:
        average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
    else:
        average_dry_masses_before_pretreatment = avg_dry_masses_before_p

    #CHANGE THIS 
    dry_masses_after_p = np.array([average_dry_masses_before_pretreatment[i]*avg_lost[i] for i in range(len(avg_lost))])
    data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment)

    process_data_keys = [
        "Dry W Milled(g)",
        "Dry W after Pretreatment(g)",
        "Glucose(g) ",
        "XMG(g)",
        "Monosacharides(g)",
        "Yield Glucose (%)",
        "Yield XMG (%)",
        "Yield Monosacharides (%)",
        "Selctivity Glucose (%)",
        "Selctivity XMG (%)",
        "Selctivity Monosacharides (%)"
    ]

    # Create dataframes
    process_data = {key: data[key] for key in process_data_keys}
    process_df = pd.DataFrame(process_data)
    process_df.index = indices

    print(f"NREL Yield Glucose: {39.54}")
    print(f"NREL Yield XMG: {25.24}")
    print(f"NREL Yield Monosacharides: {39.54+25.24}")
    print()

    return process_df


In [386]:
dry_masses_before_p_1 = np.array([2.0924, 2.0960, 2.0954, 2.0985, 2.0923, 2.0979, 2.0962, 2.0943])*0.955
avg_before_p_1 = calculate_average(reshape_into_pairs(dry_masses_before_p_1))
a_1 = avg_before_p_1
avg_lost_1 = np.array([a_1[0]*avg_lost_w_120c[0], a_1[1]*avg_lost_w_120c[2], a_1[2]*avg_lost_w_60c[0], a_1[3]*avg_lost_w_60c[2]])




In [387]:
studies = [
    {"milled_masses": np.array([2.0924, 2.0960, 2.0954, 2.0985, 2.0923, 2.0979, 2.0962, 2.0943]),
     "Glucose_samples_hplc": np.array([590520.1774, 626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333]),
     "XMG_samples_hplc": np.array([149706.8617, 159126.8929, 176542.8571, 168805.7143, 147914.2857, 172857.1429, 146857.1429, 153200.0000]),
     "glucose_yields": np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100,
     "xylmangal_values": np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100,
     "avg_lost": avg_lost_1,
     "indices": ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
     },

]

# For each study, create a dataframe and store it in a list
dataframes = []
for i, study in enumerate(studies):
    df = create_dataframe(**study)
    styled_df = df.style.background_gradient(cmap='viridis')
    dataframes.append(styled_df)
    print(f'Styled Dataframe for study {i+1}:')
    display(styled_df)



NREL Yield Glucose: 39.54
NREL Yield XMG: 25.24
NREL Yield Monosacharides: 64.78

Dataframe for study 1:
        Dry W Milled(g) Dry W after Pretreatment(g) Glucose(g)  XMG(g)  \
120C 1h            1.99                        1.95        1.16   0.27   
120C 3h            2.00                        2.07        1.37   0.32   
60C 1h             2.00                        3.13        1.92   0.46   
60C 3h             2.00                        2.18        1.24   0.29   

        Monosacharides(g) Yield Glucose (%) Yield XMG (%)  \
120C 1h              1.43              58.1          13.8   
120C 3h              1.70              68.4          16.4   
60C 1h               2.38              95.9          23.0   
60C 3h               1.54              62.1          14.9   

        Yield Monosacharides (%) Selctivity Glucose (%) Selctivity XMG (%)  \
120C 1h                     71.9                   59.4               14.1   
120C 3h                     84.9                   66.2       